In [8]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('.data/main.db')
c = conn.cursor()

In [9]:
tables = c.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [x[0] for x in c.fetchall()[1:]]
tables

['objectives', 'teams', 'tasks', 'library_values']

In [10]:
objectives, teams, tasks, values = [pd.read_sql(f'select * from {table}', conn) for table in tables]

In [11]:
objectives

,id,objective
0,0,Respond to the impacts of the pandemic within ...
1,2,Maximise use of the service and its resources
2,3,Mitigate the risks posed by the pandemic to th...


In [62]:
teams

,id,team
0,0,Pillar 0
1,1,Pillar 1
2,2,Pillar 2
3,3,Pillar 3
4,4,Pillar 4
5,5,Pillar 5
6,6,Pillar 6
7,7,Operations
8,8,Service Development Team
9,9,Management Team


In [72]:
def get_team_id(s):
    res = teams[teams.team==s].id.values
    if len(res)<1:
        return s
    else:
        return res[0]

In [13]:
tasks

,id,task,team,objective,complete
0,1,,0,3,FALSE
1,2,,0,2,FALSE
2,4,,0,0,FALSE
3,5,Conference options outside in June or online,1,3,FALSE
4,6,Review committee forward plans to potential sy...,1,2,FALSE
5,8,,1,0,FALSE
6,12,,2,0,FALSE
7,13,,2,2,FALSE
8,14,‘Discover Your Library’ to support customers t...,2,3,FALSE
9,17,,3,3,FALSE


In [128]:
xl = pd.ExcelFile('Team plan 2020-21 Excel.xlsx')
objs_lu = {'Mitigate risks to lib service ': 3,
 'Impacts of the pandemic': 0,
 'Maximise use of the service': 2}

    # ['Pillar 0',
    # 'Pillar 1',
    # 'Pillar 2',
    # 'Pillar 3',
    # 'Pillar 4',
    # 'Pillar 5',
    # 'Pillar 6',
    # 'Operations',
    # 'Service Development Team',
    # 'Management Team']

team_fix={'CL': 'Management Team',
 'SG': 'Management Team',
 'HR': 'Management Team',
 'RP': 'Management Team',
 'Ops Teams': 'Operations',
 'LMT': 'Management Team',
 'FPR': 'Management Team',
 'P6': 'Pillar 6',
 'P2': 'Pillar 2',
 'Ops Team': 'Operations',
 'Recovery Group': 'Operations',
 'P0': 'Pillar 0',
 'P4': 'Pillar 4',
 'LCSMs': 'Operations',
 'KM': 'Operations',
 'CW': 'Operations',
 'P5': 'Pillar 5',
 'CL/SG': 'Service Development Team',
 'LLL': 'Pillar 1',
 'Early Years': 'Service Development Team',
 'blank': '',
 'HR/RP': 'Management Team',
 'SDL Team': 'Service Development Team',
 'Ops ': 'Operations',
 'SM': 'Operations',
 'NM': 'Service Development Team',
 'P1': 'Pillar 1',
 'II': 'Operations',
 'SGreen': 'Operations',
 'NB': 'Service Development Team',
 'EY': 'Early Years',
 'All Teams': 'All Teams',
 'SDLs': 'Service Development Team',
 'Estates': '',
 'Equalities Group': 'Operations',
 'P3': 'Pillar 3',
 'HP': 'Operations',
 ' ': 'blank',
 'Early Years': 'Service Development Team',
 'Estates':'Operations'}

def _fix_hr(xl):
    df = pd.concat([xl.parse(sn, skiprows=2).assign(objective=objs_lu[sn]) for sn in xl.sheet_names]).reset_index().drop('index',axis=1)
    who_cols=[x for x in df.columns if 'Who' in x]
    drop_cols = [x for x in df.columns if 'Unnamed' in x]
    df = df.drop(drop_cols, axis=1)
    cols = ['What', 'Progress', 'objective'] + who_cols
    df = pd.melt(df[cols], id_vars=['What','Progress', 'objective']).rename({'value':'team', 'What':'task'}, axis=1).drop('variable',axis=1)
    df['team'] = df['team'].fillna('blank').apply(lambda x: team_fix[x] if team_fix[x] else x).replace('Early Years','Service Development Team')
    df = df[df.team!='blank']
    objectives_lookup = {id: objective for id, objective in zip(list(objectives.id), list(objectives.objective))}
    df['objective'] = df.objective.apply(lambda x: objectives_lookup[x])
    df['team_id'] = df.team.apply(get_team_id)
    def _fix(d):
        to_fix = d[d.team_id=='All Teams']
        tup = list(to_fix[['task','Progress','objective']].itertuples(index=False, name=None))[0]
        out_cols = ['task', 'Progress', 'objective', 'team', 'team_id']
        _teams = list(teams.team.unique())
        rows = []
        for t in _teams:
            rows.append(tup + (t, get_team_id(t)))
        ndf = pd.DataFrame(rows, columns=out_cols)
        return pd.concat([d[d.team_id!='All Teams'], ndf])
    df = _fix(df).reset_index().drop('index', axis=1)
    df['team_id']=pd.to_numeric(df['team_id'])
    def _objective_id(s):
        return objectives[objectives.objective==s].id.values[0]
    df['objective_id'] = df.objective.apply(_objective_id)
    out_cols = ['task','team','objective','complete']

    def _max_is_four_per_seg(df):
        # which segments have more than four - make a count column
        # return pd.merge(df, df[['objective','team','task']].groupby(['objective', 'team']).cumcount().reset_index().rename({'task':'seg_count'}, axis=1))
        df['seg_count']= df[['objective','team','task']].groupby(['objective', 'team']).cumcount()
        df['seg_count'] = df['seg_count'].apply(lambda x: x+1)
        return df
    
    # ['Pillar 0',
    # 'Pillar 1',
    # 'Pillar 2',
    # 'Pillar 3',
    # 'Pillar 4',
    # 'Pillar 5',
    # 'Pillar 6',
    # 'Operations',
    # 'Service Development Team',
    # 'Management Team']
    df = _max_is_four_per_seg(df.sort_values(['objective_id','team_id'], ascending=False))
    no_dups = df.drop_duplicates(subset='task',keep='first')
    return no_dups

_fix_hr(xl)
# xl.sheet_names

,task,Progress,objective,team,team_id,objective_id,seg_count
1,NBA – investigate how opening hours can be ext...,Open,Mitigate the risks posed by the pandemic to th...,Management Team,9,3,1
2,Here to Help newly named ‘Discover Your Librar...,Open,Mitigate the risks posed by the pandemic to th...,Management Team,9,3,2
4,Develop outreach strategy (link to IDP),Open,Mitigate the risks posed by the pandemic to th...,Management Team,9,3,3
5,Develop online enquiries service,completed,Mitigate the risks posed by the pandemic to th...,Management Team,9,3,4
10,Develop income strategy,Open,Mitigate the risks posed by the pandemic to th...,Management Team,9,3,5
12,Identify efficiencies in current budgets - Rev...,Open,Mitigate the risks posed by the pandemic to th...,Management Team,9,3,6
13,Submit inflation bid,Open,Mitigate the risks posed by the pandemic to th...,Management Team,9,3,7
14,Review use of waste and sani bins by tenants a...,Open,Mitigate the risks posed by the pandemic to th...,Management Team,9,3,8
15,Booking Live system for online booking and pay...,started,Mitigate the risks posed by the pandemic to th...,Management Team,9,3,9
16,Ensure library needs are reflected on the Infr...,started,Mitigate the risks posed by the pandemic to th...,Management Team,9,3,10
